<a href="https://colab.research.google.com/github/Brohammad/VehicleDetection/blob/main/yolobased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install opencv-python-headless cvzone

import cv2
import pickle
import cvzone
import numpy as np
import time
import os
from google.colab.patches import cv2_imshow

# Load YOLOv3 model (make sure to upload the correct files)
cfg_path = '/content/yolov3.cfg'
weights_path = '/content/yolov3.weights'

# Check if files exist
assert os.path.exists(cfg_path), "Config file not found"
assert os.path.exists(weights_path), "Weights file not found"

# Load YOLOv3 model
net = cv2.dnn.readNetFromDarknet(cfg_path, weights_path)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load the class labels
with open("/content/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Load parking slot positions from file
with open('/content/CarParkPos', 'rb') as f:
    posList = pickle.load(f)

# Define parking slot size
width, height = 107, 48

# Video capture
cap = cv2.VideoCapture('/content/carPark.mp4')  # Replace with your video path

# Function to check parking spaces
def checkParkingSpace(imgPro):
    spaceCounter = 0
    for pos in posList:
        x, y = pos
        imgCrop = imgPro[y:y + height, x:x + width]
        count = cv2.countNonZero(imgCrop)

        color = (0, 255, 0) if count < 900 else (0, 0, 255)
        thickness = 5 if count < 900 else 2
        spaceCounter += (count < 900)

        cv2.rectangle(img, pos, (pos[0] + width, pos[1] + height), color, thickness)
        cvzone.putTextRect(img, str(count), (x, y + height - 3), scale=1,
                           thickness=2, offset=0, colorR=color)

    cvzone.putTextRect(img, f'Free: {spaceCounter}/{len(posList)}', (100, 50), scale=3,
                           thickness=5, offset=20, colorR=(0, 200, 0))

# Function to detect vehicles using YOLO
def detect_vehicles(frame):
    height, width, channels = frame.shape

    # Prepare the frame for YOLO model (Blob conversion)
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    # Process YOLO model outputs
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Confidence threshold
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Coordinates for rectangle
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-maxima suppression to avoid multiple boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw bounding boxes for detected vehicles
    for i in range(len(boxes)):
        if i in indices:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    return frame

# Process video and detect parking spaces
frame_skip = 5  # Display every 5th frame
frame_count = 0

while True:
    success, img = cap.read()
    if not success:
        break

    frame_count += 1
    if frame_count % frame_skip == 0:  # Only process selected frames
        imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        imgBlur = cv2.GaussianBlur(imgGray, (3, 3), 1)
        imgThreshold = cv2.adaptiveThreshold(imgBlur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                             cv2.THRESH_BINARY_INV, 25, 16)
        imgMedian = cv2.medianBlur(imgThreshold, 5)
        kernel = np.ones((3, 3), np.uint8)
        imgDilate = cv2.dilate(imgMedian, kernel, iterations=1)

        # Detect vehicles in the frame using YOLO
        img = detect_vehicles(img)

        # Check parking space status
        checkParkingSpace(imgDilate)

        # Display the frame with parking slot overlay
        cv2_imshow(img)

        # Optional: Control frame rate with sleep
        time.sleep(0.1)

# Release resources
cap.release()
cv2.destroyAllWindows()